In [2]:
import sys
sys.path.append('../src')

import importlib
import numpy as np
from sklearn.covariance import empirical_covariance

import policies 

# Covariance Constrained Policy Testing

Testing the covariance constrained policy. Initially, this was giving a lot of violations. Initially thought that the problem might just not be feasible but after inspection it was. I switched the solver being used by cvxpy to Gurobi, which resolved the issue. 

The initial line in this, %%capture, redirects stdout from printing out the output. This is much nicer than redirecting the stdout manually, which doesn't work as anticipated inside of jupyter notebooks. Note: if your code is multithreaded, this no longer works for some reason. 

In [24]:
%%capture

importlib.reload(policies)

n = 50
n_features = 6
pred_dim = 6

np.random.seed(30)

# Gaussian xs, with true labels having a different linear relationship with xs in each coordinate
xs = np.random.normal(size=(n, n_features))
slopes = np.random.uniform(size = n_features)
ys = np.multiply(xs, slopes)

def meta_model(coord, slopes):
    def model(xs):
        preds = np.random.normal(size=(n, pred_dim))
        true_ys = np.multiply(xs, slopes)
        preds[:,coord] = true_ys[:,coord]
        return preds
    return model

model = meta_model(0, slopes)

alpha = 0.5
policy = policies.VarianceConstrained(pred_dim, model, 0.1, alpha, ys)
out = policy.run_given_preds(ys);

In [25]:
out

array([[5.15141811e-11, 3.51809664e-08, 5.18379737e-09, 9.99999931e-01,
        1.22299442e-08, 1.59185196e-08],
       [3.35132570e-09, 3.00032070e-01, 2.10536594e-07, 4.45967594e-08,
        6.99966277e-01, 1.39371189e-06],
       [1.54675785e-10, 8.08830045e-09, 3.70902685e-08, 9.99999933e-01,
        2.79591850e-09, 1.86735786e-08],
       [3.82132833e-11, 1.64369476e-07, 2.74086899e-08, 5.87678114e-10,
        5.85162433e-11, 9.99999808e-01],
       [2.63960873e-09, 5.79476935e-06, 2.99116728e-01, 7.89413007e-08,
        7.00877231e-01, 1.64842111e-07],
       [1.31010820e-11, 3.33703153e-09, 9.99999994e-01, 9.36494767e-11,
        1.77207184e-11, 2.48174225e-09],
       [3.53814336e-01, 4.64776875e-10, 4.74456930e-10, 6.88193260e-10,
        6.46185662e-01, 4.75952183e-10],
       [2.09143389e-08, 1.68772738e-07, 8.53274549e-08, 2.98305681e-01,
        7.01693923e-01, 1.20439127e-07],
       [1.61385505e-10, 5.98201381e-10, 1.04287676e-09, 2.98306046e-01,
        7.01693951e-01, 

Sanity check: 

1. Does each row sum to 1? Yes! Or at least, up to a tolerance, since there will be small floating point errors.

In [26]:
tolerance = 1e-3

print(np.sum(out, axis=1))
print("Violations: ", n-sum((np.isclose(np.sum(out, axis=1), np.ones(50), atol=tolerance))))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]
Violations:  0


2. Is each constraint bounded by 0 and 1?

In [27]:
print(f"Number of allocations greater than 1: {np.sum(out - tolerance > 1)}")
print(f"Number of allocations less than 0: {np.sum(out+tolerance < 0)}")

Number of allocations greater than 1: 0
Number of allocations less than 0: 0


3. Are the variance conditions being approx satisfied? I wrote yes here before but it seems like no?

In [28]:
cov = empirical_covariance(ys, assume_centered=False)

viol = np.zeros(len(out))
for i in range(len(out)):
    viol[i] = np.matmul(np.matmul(out[i], cov), np.transpose(out[i]))

print(f"Max violation: {alpha}")
print(f"Number of variance constraints which violate the max allowed variance: {sum(viol > alpha+0.1)}")

Max violation: 0.5
Number of variance constraints which violate the max allowed variance: 0


3. Is the covariance matrix actually measuring the correct thing? A sanity check that the diagonal of the matrix is equal to empirical variance.


In [29]:
print(f"Variances of each coordinate of the ys: \n {np.var(ys, axis=0)}")

print(f"Diagonal of the covariance matrix: \n {np.diagonal(cov)}")

Variances of each coordinate of the ys: 
 [0.82798883 0.00104122 0.00179195 0.10912913 1.02202326 0.00313415]
Diagonal of the covariance matrix: 
 [0.82798883 0.00104122 0.00179195 0.10912913 1.02202326 0.00313415]


4. Are all the constraints convex and are things feasible?

If C is positive semi-definite, then xCx^T <= val is a convex constraint on R^n. Maybe something went wrong in calculation of C? 

If the eigenvalues of C are positive, then C must be positive semi-definite. Here we see this is the case:

In [30]:
np.linalg.eigvals(cov) > 0

array([ True,  True,  True,  True,  True,  True])

If the only constraint was covariance, xCx^T will be bounded by the eigenvalues of covariance matrix, so this is a good spot check for how feasible things are

In [31]:
np.linalg.eigvals(cov)

array([1.04541420e+00, 8.10321387e-01, 1.04229047e-01, 2.73097313e-03,
       8.41962782e-04, 1.57097101e-03])

# Linear Constraint Policy Testing

In [10]:
importlib.reload(policies)

n = 50
n_features = 6
pred_dim = 6

np.random.seed(30)

# Gaussian xs, with true labels having a different linear relationship with xs in each coordinate
xs = np.random.normal(size=(n, n_features))
slopes = np.random.uniform(size = n_features)
ys = np.multiply(xs, slopes)

def meta_model(coord, slopes):
    def model(xs):
        preds = np.random.normal(size=(n, pred_dim))
        true_ys = np.multiply(xs, slopes)
        preds[:,coord] = true_ys[:,coord]
        return preds
    return model

model = meta_model(0, slopes)

linear_constraint = np.array([[1,1,1,0,0,0], [0,0,1,1,0,0]])
max_val = np.array([0.5,0.2])
gran = 0.1

policy = policies.Linear(pred_dim, model, gran, linear_constraint, max_val)
out = policy.run_given_preds(ys);

In [11]:
np.sum(np.multiply(out, linear_constraint[0])[:,[0,1,2]], axis=1) <= max_val[0]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [12]:
np.sum(np.multiply(out, linear_constraint[0])[:,[2,3]], axis=1) <= max_val[1]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [8]:
linear_constraint[1]

array([0, 0, 1, 1, 0, 0])